# Write ATS input file

We now generate three input files -- two for spinup (steadystate solution and cyclic steadystate solution) and one for transient runs.

* Input files: ATS xml files
  - `{WATERSHED_NAME}_spinup-steadystate.xml` the steady-state solution based on uniform application of mean rainfall rate
  - `{WATERSHED_NAME}_spinup-cyclic_steadystate.xml` the cyclic steady state based on typical years
  - `{WATERSHED_NAME}_transient.xml` the forward model

In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os, yaml, pickle, datetime
import pandas as pd

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s: %(message)s')

# ats_input_spec library, to be moved to amanzi_xml
import ats_input_spec
import ats_input_spec.public
import ats_input_spec.io

# amanzi_xml, included in AMANZI_SRC_DIR/tools/amanzi_xml
import amanzi_xml.utils.io as aio
import amanzi_xml.utils.search as asearch
import amanzi_xml.utils.errors as aerrors

In [14]:
import modvis
from modvis import ats_xml

In [15]:
watershed_name = 'RedButteCreek'
config_fname = f'../../data/examples/{watershed_name}/processed/config.yaml'
watershed_latitude = 39 # the average latitude of the watershed in degree, used for calculating incident radiation

## load configuration

In [16]:
# Load the dictionary from the file
with open(config_fname, 'r') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [17]:
config['spinup_steadystate_rundir'] = os.path.join('1-spinup_steadystate')
config['spinup_cyclic_rundir'] = os.path.join('2-spinup_cyclic')
config['transient_rundir'] = os.path.join('3-transient')

config['spinup_steadystate_template'] = os.path.join('..', '..', 'model', 'inputs', 'spinup_steadystate-template.xml')
config['spinup_cyclic_template'] = os.path.join('..', '..', 'model', 'inputs', 'spinup_cyclic-template.xml')
config['transient_template'] = os.path.join('..', '..', 'model', 'inputs', 'transient-template.xml')

config['watershed_specific_xml'] = os.path.join('..', '..', 'model', 'inputs', f'{watershed_name}_specific.xml')
config['spinup_steadystate_xml'] = os.path.join('..', '..', 'model', 'inputs', f'{watershed_name}_spinup_steadystate.xml')
config['spinup_cyclic_xml'] = os.path.join('..', '..', 'model', 'inputs', f'{watershed_name}_spinup_cyclic.xml')
config['transient_xml'] = os.path.join('..', '..', 'model', 'inputs', f'{watershed_name}_transient.xml')

##---CHANGE THIS!-------##
config['latitude [deg]'] = watershed_latitude # latitude of watershed in degree, used to determine incident radiation

In [18]:
config

{'LAI_filename': '../../data/examples/RedButteCreek/processed/watershed_lai_raw.h5',
 'LAI_typical_filename': '../../data/examples/RedButteCreek/processed/watershed_lai_typical.h5',
 'catchment_labels': ['RedButteCreek'],
 'config_file': '../../data/examples/RedButteCreek/processed/config.yaml',
 'daymet_filename': '../../data/examples/RedButteCreek/processed/watershed_daymet_raw.h5',
 'daymet_typical_filename': '../../data/examples/RedButteCreek/processed/watershed_daymet_typical.h5',
 'end_date': '2020-10-01',
 'labeled_sets': {'RedButteCreek': {'entity': 'CELL', 'setid': 10000}},
 'mean_precip [m s^-1]': 2.3817113055336835e-08,
 'mesh_filename': '../../data/examples/RedButteCreek/processed/watershed_mesh.exo',
 'nlcd_indices': [9, 10],
 'nlcd_labels': ['Savannas', 'Grasslands'],
 'nyears_cyclic_steadystate': 4,
 'origin_date': '1980-1-1',
 'side_sets': {'Grasslands': {'setid': 10},
  'RedButteCreek boundary': {'setid': 10002},
  'RedButteCreek outlet': {'setid': 10003},
  'RedButteC

In [19]:
# nlcd_indices = config['nlcd_indices']
nlcd_labels = config['nlcd_labels']
subcatchment_labels = config['catchment_labels']
ls = config['labeled_sets']
ss = config['side_sets']
mean_precip = config['mean_precip [m s^-1]']
start_date = config['start_date']
end_date = config['end_date']

In [20]:
# load subsurface properties
subsurface_props = pd.read_csv(config['subsurface_properties_filename'], index_col='ats_id')

## Write input files

Replace template files with generated watershed specific properties. This also sets the start and end date of the simulations, and creates directories for each run.

- `{name}_spinup_steadystate.xml`: For the first file, we load a spinup template and write the needed quantities into that file, saving it to the appropriate run directory.  Note there is no DayMet or land cover or LAI properties needed for this run.  The only property that is needed is the domain-averaged, mean annual rainfall rate.  We then take off some for ET (note too wet spins up faster than too dry, so don't take off too much...).

- `{name}_spinup_cyclic.xml`: For the second file, we load a transient run template.  This file needs the basics, plus DayMet and LAI as the "typical year data".  Also we set the run directory that will be used for the steadystate run.

- `{name}_transient.xml`: For the third file, we load a transient run template as well.  This file needs the basics, DayMet with the actual data, and we choose for this run to use the MODIS typical year.  MODIS is only available for 2002 on, so if we didn't need 1980-2002 we could use the real data, but for this run we want a longer record.

In [21]:
# create a steady-state run
ats_xml.write_spinup_steadystate(config, mean_precip = mean_precip,
                            subsurface_props=subsurface_props, nlcd_labels = nlcd_labels,
                             labeled_sets = ls, side_sets = ss,
                             subcatchment_labels=subcatchment_labels)

# make sure the cyclic ends near Oct. 1. Default cyclic period is 10 years
ats_xml.write_transient(config,             
                start_date = start_date, end_date=end_date, subsurface_props=subsurface_props,
                nlcd_labels = nlcd_labels, labeled_sets = ls, side_sets = ss,
                subcatchment_labels=subcatchment_labels, 
                nyears_cyclic_steadystate = 4,
                cyclic_steadystate=True, 
               )

# create the fully-heterogeneous runs
ats_xml.write_transient(config,             
                start_date = start_date, end_date=end_date, subsurface_props=subsurface_props,
                nlcd_labels = nlcd_labels, labeled_sets = ls, side_sets = ss,
                subcatchment_labels=subcatchment_labels, 
                cyclic_steadystate=False, 
               )

2025-04-24 19:49:30,143 - root - INFO: Writing spinup steadystate xml: ../../model/inputs/RedButteCreek_spinup_steadystate.xml
/home/jovyan/ats/ats_input_spec/ats_input_spec/io.py:43: UserWarning: Creating an incomplete XML object, missing entries!
  warnings.warn('Creating an incomplete XML object, missing entries!')
2025-04-24 19:49:30,174 - root - INFO: Directory '../../model/1-spinup_steadystate' created successfully.
2025-04-24 19:49:30,176 - root - INFO: Writing spinup_cyclic xml: ../../model/inputs/RedButteCreek_spinup_cyclic.xml
2025-04-24 19:49:30,506 - root - INFO: Directory '../../model/2-spinup_cyclic' created successfully.
2025-04-24 19:49:30,507 - root - INFO: Writing transient xml: ../../model/inputs/RedButteCreek_transient.xml
2025-04-24 19:49:30,550 - root - INFO: Directory '../../model/3-transient' created successfully.


In [22]:
with open(config_fname, 'w') as f:
    yaml.dump(config, f)